In [1]:
import data
from utils import distances

symale = [ 2 if i > 7 else 1 for i in data.SYAMALE_MEENAKSHI["melody"]]
orig = [ i   for i in data.AH_VOUS_ORIGINAL["melody"]][1:-1]
orig = [ 2 if i > 7 else 1 for i in orig]
moz_thm = []


print("LCS: {}, ED: {}, dETC: {}, HD: {}".format(distances.LCSubSeq(symale, orig, len(symale), len(orig)),
                                                 distances.editDist(symale, orig, len(symale), len(orig)),
                                                 distances.etcDist(symale, orig),
                                                 distances.hammingDist(symale, orig, len(symale), len(orig))))

LCS: 11, ED: 27, dETC: 10.0, HD: 0.4807692307692308


In [8]:
import ETC

def get_etc_modified_data(dict):
    ret = []
    for ind in range(len(dict["melody"])):
        if dict["melody"][ind]!= 99999:
            if dict["melody"][ind] > 7:
                ret.append(2)
            else:
                ret.append(1)
    return ret

m1 = get_etc_modified_data(data.AH_VOUS_ORIGINAL)
m2 = get_etc_modified_data(data.SYAMALE_MEENAKSHI)
m3 = get_etc_modified_data(data.MOZART_THEME)
m4 = get_etc_modified_data(data.MOZART_VARIATION1)

etc1  = ETC.compute_1D(m1, order=2, verbose=False, truncate=True)["ETC1D"]
etc2  = ETC.compute_1D(m2, order=2, verbose=False, truncate=True)["ETC1D"]
etc3  = ETC.compute_1D(m3, order=2, verbose=False, truncate=True)["ETC1D"]
etc4  = ETC.compute_1D(m4, order=2, verbose=False, truncate=True)["ETC1D"]


print("LCS: {}, ED: {}, dETC: {}, HD: {}".format(distances.LCSubSeq(m1, m2, len(m1), len(m2)),
                                                 distances.editDist(m1, m2, len(m1), len(m2)),
                                                 distances.etcDist(m1, m2),
                                                 distances.hammingDist(m1, m2, len(m1), len(m2))))


print("LCS: {}, ED: {}, dETC: {}, HD: {}".format(distances.LCSubSeq(m1, m3, len(m1), len(m3)),
                                                 distances.editDist(m1, m3, len(m1), len(m3)),
                                                 distances.etcDist(m1, m3),
                                                 distances.hammingDist(m1, m3, len(m1), len(m3))))


print("LCS: {}, ED: {}, dETC: {}, HD: {}".format(distances.LCSubSeq(m4, m3, len(m4), len(m3)),
                                                 distances.editDist(m4, m3, len(m4), len(m3)),
                                                 distances.etcDist(m4, m3),
                                                 distances.hammingDist(m4, m3, len(m4), len(m3))))

etc1, etc2, etc3, etc4

LCS: 11, ED: 27, dETC: 11.0, HD: 0.5094339622641509
LCS: 53, ED: 0, dETC: 5.0, HD: 0.0
LCS: 53, ED: 0, dETC: 5.0, HD: 0.0


(21, 18, 21, 21)